In [211]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [212]:
def get_data_and_first_cleaning():
    rooth_path = '../rawdata/training/'
    failures_df = pd.read_csv(rooth_path + 'wind-farm-1-failures-training.csv')
    locations_df = pd.read_csv(rooth_path + "wind-farm-1-locations.csv")
    logs_df = pd.read_csv(rooth_path + 'wind-farm-1-logs-training.csv')
    metmast_df = pd.read_csv(rooth_path + 'wind-farm-1-metmast-training.csv', sep=';')
    signals_df = pd.read_csv(rooth_path + 'wind-farm-1-signals-training.csv', sep=';')
    power_df = pd.read_csv(rooth_path + 'Power_curve.csv', sep=',')
#     costs_df = pd.read_csv(rooth_path + 'HTW_Costs.csv')
    
    # pass all the Timestamps to date-time format
    failures_df['Timestamp'] = pd.to_datetime(failures_df['Timestamp'])
    logs_df['Timestamp'] = pd.to_datetime(logs_df['TimeDetected'])
    logs_df = logs_df.drop(columns=['TimeDetected','UnitTitleDestination'])
    metmast_df['Timestamp'] = pd.to_datetime(metmast_df['Timestamp'])
    signals_df['Timestamp'] = pd.to_datetime(signals_df['Timestamp'])
     
    # Drop Columns with std=0 by DA
    drop_features_metmast = ['Anemometer1_Offset', 'Anemometer2_Freq', 'Anemometer2_Offset', 'Min_Raindetection',
                             'Avg_Raindetection', 'Anemometer1_CorrGain', 'Anemometer1_CorrOffset',
                             'Anemometer2_CorrGain', 'Anemometer2_CorrOffset', 'DistanceAirPress', 
                             'AirRessureSensorZeroOffset']
    metmast_df = metmast_df.drop(columns=drop_features_metmast)
    
    drop_features_signals = ['Prod_LatestAvg_ActPwrGen2', 'Prod_LatestAvg_ReactPwrGen2']
    signals_df = signals_df.drop(columns=drop_features_signals)
    
    failures_df = failures_df.drop(columns='Remarks')
    
    return failures_df, locations_df, logs_df, metmast_df, signals_df, power_df

In [213]:
failures_df, locations_df, logs_df, metmast_df, signals_df, power_df = get_data_and_first_cleaning()

In [214]:
failures_df.head()

,Turbine_ID,Component,Timestamp
0,T11,GENERATOR,2016-03-03 19:00:00+00:00
1,T06,HYDRAULIC_GROUP,2016-04-04 18:53:00+00:00
2,T07,GENERATOR_BEARING,2016-04-30 12:40:00+00:00
3,T09,GENERATOR_BEARING,2016-06-07 16:59:00+00:00
4,T07,TRANSFORMER,2016-07-10 03:46:00+00:00


In [215]:
# Create functions to make datasets for each turbine
def turbine(turbine_id, df):
    new_df = df[df['Turbine_ID'] == turbine_id].sort_index(ascending=True)
    return new_df

# Function to find str in columns of df
def component(component, col):
    pair_comp_col=[]
    for i in col:
        if component in i:
            pair_comp_col.append(i)
    return pair_comp_col

# Function to analyse by component
def component_df(component, df):
    df_component = df[df['Component'] == component].sort_index(ascending=True)
    return df_component

In [216]:
def prepare_data(df, strategy='mean'):
    if strategy == 'mean':
        df = df.groupby(by=['Timestamp', 'Turbine_ID']).mean()#.sort_values(['Turbine_ID', 'Timestamp'])
    else:
        df = df.grouby(by=['Timestamp', 'Turbine_ID']).max()#.sort_values(['Turbine_ID', 'Timestamp'])
    
    # Fill na by interpolate
    df = df.interpolate().reset_index()
        
    return df

In [217]:
telemetry = prepare_data(signals_df)

In [218]:
telemetry

,Timestamp,Turbine_ID,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,...,Grd_Prod_PsbleInd_Avg,Grd_Prod_PsbleInd_Max,Grd_Prod_PsbleInd_Min,Grd_Prod_PsbleInd_Std,Grd_Prod_PsbleCap_Avg,Grd_Prod_PsbleCap_Max,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg
0,2016-01-01 00:00:00+00:00,T01,1277.4,1226.1,1249.0,9.0,41.0,58.0,59.0,58.0,...,-144.4,0.0,-584.5,157.1,144.4,584.5,0.0,157.1,37.0,218.5
1,2016-01-01 00:00:00+00:00,T06,1270.0,1232.8,1248.5,6.8,42.0,51.0,52.0,53.0,...,-307.1,0.0,-880.6,180.0,307.1,880.6,0.0,180.0,35.0,204.6
2,2016-01-01 00:00:00+00:00,T07,1317.5,1229.5,1254.9,13.8,41.0,62.0,62.0,61.0,...,-536.5,0.0,-1000.0,338.9,536.5,1000.0,0.0,338.9,39.0,197.3
3,2016-01-01 00:00:00+00:00,T09,1376.7,1234.3,1272.0,24.7,40.0,58.0,57.0,58.0,...,-861.6,-224.5,-1000.0,185.5,861.6,1000.0,224.5,185.5,33.0,214.0
4,2016-01-01 00:00:00+00:00,T11,1339.4,1233.3,1270.9,18.8,42.0,59.0,61.0,60.0,...,-973.4,-752.6,-1000.0,56.5,973.4,1000.0,752.6,56.5,38.0,206.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434110,2017-09-01 00:00:00+00:00,T01,1299.0,1222.1,1255.3,12.4,45.0,64.0,64.0,64.0,...,-822.6,-314.2,-1000.0,203.6,822.6,1000.0,314.2,203.6,44.0,109.5
434111,2017-09-01 00:00:00+00:00,T06,1284.2,1233.4,1252.4,8.8,49.0,63.0,64.0,65.0,...,-725.4,-171.8,-1000.0,272.2,725.4,1000.0,171.8,272.2,45.0,84.1
434112,2017-09-01 00:00:00+00:00,T07,1292.0,1232.5,1251.9,9.6,48.0,66.0,67.0,66.0,...,-948.8,-745.0,-1000.0,74.2,948.8,1000.0,745.0,74.2,43.0,90.1
434113,2017-09-01 00:00:00+00:00,T09,1268.4,1240.1,1251.9,6.0,44.0,62.0,61.0,62.0,...,-900.6,-340.3,-1000.0,121.6,900.6,1000.0,340.3,121.6,40.0,105.0


In [219]:
failures_df = pd.get_dummies(failures_df, columns=["Component"])

In [220]:
failures_df.head()

,Turbine_ID,Timestamp,Component_GEARBOX,Component_GENERATOR,Component_GENERATOR_BEARING,Component_HYDRAULIC_GROUP,Component_TRANSFORMER
0,T11,2016-03-03 19:00:00+00:00,0,1,0,0,0
1,T06,2016-04-04 18:53:00+00:00,0,0,0,1,0
2,T07,2016-04-30 12:40:00+00:00,0,0,1,0,0
3,T09,2016-06-07 16:59:00+00:00,0,0,1,0,0
4,T07,2016-07-10 03:46:00+00:00,0,0,0,0,1


In [221]:
#Features to create each DF
def create_features(df):
    time_id = ['Timestamp', 'Turbine_ID']
    pair_hyd = component('Hyd', df.columns)
    pair_trafo = component('Trafo', df.columns)
    pair_gear = component('Gear', df.columns)
    pair_gen = component('Gen', df.columns)
    pair_rot = component('Rtr', df.columns)
    pair_amb = component('Amb', df.columns)
    pair_blds = component('Blds', df.columns)
    pair_cont = component('Cont', df.columns)
    pair_nac = component('Nac', df.columns)
    pair_spin = component('Spin', df.columns)
    pair_bus = component('Busbar', df.columns)
    pair_vol = component('Volt', df.columns)
    
    #Create DF for each component
    df_generator = df[time_id + pair_gen + pair_rot + pair_amb + pair_blds + pair_cont + pair_nac + pair_spin + pair_bus + pair_hyd]
    df_gen_bear = df[time_id + pair_gen + pair_rot + pair_amb + pair_blds + pair_cont + pair_nac + pair_spin + pair_bus + pair_hyd]
    df_transformer = df[time_id + pair_trafo + pair_rot + pair_amb + pair_blds + pair_cont + pair_nac + pair_spin + pair_bus + pair_vol] 
    df_hydraulic = df[time_id + pair_hyd + pair_rot + pair_amb + pair_blds + pair_cont + pair_nac + pair_spin + pair_bus] 
    df_gearbox = df[time_id + pair_gear + pair_rot + pair_amb + pair_blds + pair_cont + pair_nac + pair_spin + pair_bus + pair_hyd]
    
    return df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox

In [222]:
df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox = create_features(telemetry)

In [241]:
# def create_df_failures(df):
#     time_id = ['Timestamp', 'Turbine_ID']
#     pair_gen_fail = component('', df.columns)
#     pair_trafo = component('Trafo', df.columns)
#     pair_gear = component('Gear', df.columns)
#     pair_gen = component('Gen', df.columns)
#     pair_rot = component('Rtr', df.columns)
failures_df.loc[:,['Turbine_ID','Timestamp']]

,Turbine_ID,Timestamp
0,T11,2016-03-03 19:00:00+00:00
1,T06,2016-04-04 18:53:00+00:00
2,T07,2016-04-30 12:40:00+00:00
3,T09,2016-06-07 16:59:00+00:00
4,T07,2016-07-10 03:46:00+00:00
5,T06,2016-07-11 19:48:00+00:00
6,T01,2016-07-18 02:10:00+00:00
7,T06,2016-07-24 17:01:00+00:00
8,T09,2016-08-22 18:25:00+00:00
9,T07,2016-08-23 02:21:00+00:00


In [249]:
failures_generator = failures_df.loc[:,['Timestamp', 'Turbine_ID', 'Component_GENERATOR']]
failures_gen_bear = failures_df.loc[:,['Timestamp', 'Turbine_ID', 'Component_GENERATOR_BEARING']]
failures_hyd = failures_df.loc[:,['Timestamp', 'Turbine_ID', 'Component_HYDRAULIC_GROUP']]
failures_transformer = failures_df.loc[:,['Timestamp', 'Turbine_ID', 'Component_TRANSFORMER']]
failures_gearbox = failures_df.loc[:,['Timestamp', 'Turbine_ID', 'Component_GEARBOX']]

In [250]:
def timestamp_round_down(df, time_column='Timestamp'):
    'Arredondar os intervalos de tempo para os 10 minutos anteriores'
    df[time_column] = df.apply(lambda x: x[time_column] - datetime.timedelta(minutes=x[time_column].minute % -10,seconds=x[time_column].second, microseconds=x[time_column].microsecond),axis=1)
    return df

In [251]:
failures_generator = timestamp_round_down(failures_generator, time_column='Timestamp')
failures_gen_bear = timestamp_round_down(failures_gen_bear, time_column='Timestamp')
failures_hyd = timestamp_round_down(failures_hyd, time_column='Timestamp')
failures_transformer = timestamp_round_down(failures_transformer, time_column='Timestamp')
failures_gearbox = timestamp_round_down(failures_gearbox, time_column='Timestamp')

In [252]:
generator_final = df_generator.merge(failures_generator, on=['Timestamp','Turbine_ID'], how='outer')

In [253]:
generator_final

,Timestamp,Turbine_ID,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,...,Cont_Hub_Temp_Avg,Cont_VCP_Temp_Avg,Cont_VCP_ChokcoilTemp_Avg,Cont_VCP_WtrTemp_Avg,Nac_Temp_Avg,Nac_Direction_Avg,Spin_Temp_Avg,Grd_Busbar_Temp_Avg,Hyd_Oil_Temp_Avg,Component_GENERATOR
0,2016-01-01 00:00:00+00:00,T01,1277.4,1226.1,1249.0,9.0,41.0,58.0,59.0,58.0,...,28.0,43.0,91.0,39.0,28.0,218.5,20.0,38.0,30.0,NaN
1,2016-01-01 00:00:00+00:00,T06,1270.0,1232.8,1248.5,6.8,42.0,51.0,52.0,53.0,...,28.0,38.0,96.0,38.0,29.0,204.6,20.0,37.0,30.0,NaN
2,2016-01-01 00:00:00+00:00,T07,1317.5,1229.5,1254.9,13.8,41.0,62.0,62.0,61.0,...,28.0,39.0,91.0,38.0,27.0,197.3,20.0,36.0,32.0,NaN
3,2016-01-01 00:00:00+00:00,T09,1376.7,1234.3,1272.0,24.7,40.0,58.0,57.0,58.0,...,27.0,39.0,93.0,38.0,29.0,214.0,19.0,36.0,44.0,NaN
4,2016-01-01 00:00:00+00:00,T11,1339.4,1233.3,1270.9,18.8,42.0,59.0,61.0,60.0,...,28.0,42.0,90.0,38.0,27.0,206.9,20.0,39.0,30.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434114,2017-09-01 00:00:00+00:00,T11,1266.3,1234.3,1250.6,5.1,46.0,60.0,61.0,61.0,...,33.0,45.0,80.0,39.0,31.0,96.8,28.0,41.0,39.0,NaN
434115,2016-03-03 19:00:00+00:00,T11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
434116,2016-07-11 19:50:00+00:00,T06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
434117,2017-08-11 13:20:00+00:00,T01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [224]:
# def timestamp_round_down(df, time_column='Timestamp'):
#     'Arredondar os intervalos de tempo para os 10 minutos anteriores'
#     df[time_column] = df.apply(lambda x: x[time_column] - datetime.timedelta(minutes=x[time_column].minute % -10,seconds=x[time_column].second, microseconds=x[time_column].microsecond),axis=1)
#     return df

In [225]:
# def time_transform(df, time_column='Timestamp'):
#     'Transformar as colunas referentes a tempo no data typw tempo'
# #     df[time_column] = pd.to_datetime(df[time_column].str[:19])
#     # df[time_column] = df[time_column].dt.tz_localize('GMT')
#     df[time_column] = df[time_column].dt.tz_convert(None)
#     return df

In [226]:
# failures_df.dtypes

In [227]:
# telemetry.dtypes

In [228]:
# failures_df = time_transform(failures_df)
# telemetry = time_transform(telemetry)

In [229]:
# temp = timestamp_round_down(failures_df, 'Timestamp')

In [230]:
# final = telemetry.merge(temp, on=['Timestamp', 'Turbine_ID'], how='outer')

In [231]:
# final